## Robust Principal Component Analysis (RPCA)

> Robust PCA is a technique for decomposing a data matrix into two terms: a low-rank matrix capturing the characteristics of interest and a sparse matrix containing noise that captures corrupted the data.

Standard PCA can be thought of as a maximization problem (maximizing the preserved variance of the data) or a minimization problem (minimizing the squared errors between data and their projection on the principal axes). Using this approach, the solution will be strongly skewed towards the outliers as they use an $L2$-norm, making the solution sensitive to outliers (not robust). That is to say, PCA is a fragile approach that will fail when your data is either corrupted or incomplete. An answer to this dilemma was developed by Candes, Li, Ma, and Wright in their 2009 paper _"Robust Principal Component Analysis?"_ (see __[1]__ in __References__).

Like Sparse PCA (see my [notebook on Sparse PCA](../sparsePCA/sparsePCA.ipynb)), there are multiple approaches to solving this problem. This notebook will only set up the problem and refer the reader to specifics on theory and derivation.

Given a data matrix $X \in R^{n \times m}$, we wish to decompose it as the sum of a low-rank matrix $L$ and a sparse matrix $S$: $$X = L+S$$

Here, $X$ contains our corrupted data, $L$ is of low rank and contains our corrected data, and $S$ will hold all the outlier measurements. Given this decomposition, $L$ will be a new dataset we can work with that will be well approximated by standard PCA or other machine learning algorithms. Mathematically, this (idealized) problem can be written as $$\underset{L, S}{min}\;\;\{rank(L)+||S||_0\}\;\;s.t.\;\;X = L+S$$

This problem is intractable as neither component is convex and there is no guarantee to a solution. To solve this, a convex relaxation is introduced using proxies for the actual operators of interest. The rank is replaced by the _nuclear norm_ (the sum of singular values - more zero singular values means lower rank) and the $0$-norm is replaced by the $1$-norm (sum of absolute values).

$$\underset{L, S}{min}\;\;\{||L||_{*}+\lambda||S||_1\}\;\;s.t.\;\;X = L+S$$


The above formulation's solution converges to the idealized formulation with high probability providing $\lambda =  \frac{1}{\sqrt{max(n,m)}}$ (see __[2]__).

From this point, the method of Augmented Lagrange Multipliers can be applied (__[3]__) to find the desired decomposition. Per __[2]__, you can also utilize the Alternating Directions Method (ADM) by applying a shrinkage operator and a Singular Value Threshold (SVT) operator iteratively. The following code block defining the functions `shrink`, `SVT`, and `RPCA` is provided by __[2]__ and all credit goes to Steven L. Brunton, J. Nathan Kutz, and Daniel Dylewsky (for translating the original MATLAB code to Python) for these functions. We apply the results from the aformentioned functions to a problem involving denoising handwritten digits.

In [3]:
# RPCA from Steven L. Brunton; J. Nathan Kutz (2019)
# Available at https://databookuw.com/
import numpy as np

def shrink(X,tau):
    Y = np.abs(X)-tau
    return np.sign(X) * np.maximum(Y,np.zeros_like(Y))
    
def SVT(X,tau):
    U,S,VT = np.linalg.svd(X,full_matrices=0)
    out = U @ np.diag(shrink(S,tau)) @ VT
    return out
    
def RPCA(X):
    n1,n2 = X.shape
    mu = n1*n2/(4*np.sum(np.abs(X.reshape(-1))))
    lambd = 1/np.sqrt(np.maximum(n1,n2))
    thresh = 10**(-7) * np.linalg.norm(X)
    
    S = np.zeros_like(X)
    Y = np.zeros_like(X)
    L = np.zeros_like(X)
    count = 0
    while (np.linalg.norm(X-L-S) > thresh) and (count < 1000):
        L = SVT(X-S+(1/mu)*Y,1/mu)
        S = shrink(X-L+(1/mu)*Y,lambd/mu)
        Y = Y + mu*(X-L-S)
        count += 1
    return L,S

__Application__

<u>__References__<u>

__[1]__ Emmanuel J. Candes; Xiaodong Li; Yi Ma; John Wright (2009). "Robust Principal Component Analysis?". Journal of the ACM. 58 (3): 1–37

__[2]__ Steven L. Brunton; J. Nathan Kutz (2019). "Data Driven Science and Engineering: Machine Learning, Dynamical Systems, and Control". Section 3.7 

__[3]__ Hestenes, M. R. (1969). "Multiplier and gradient methods". Journal of Optimization Theory and Applications. 4 (5): 303–320.